# 1. Implementing your own Shazam
###Import libraries:

In [2]:
import numpy as np      
import matplotlib.pyplot as plt 
import scipy.io.wavfile 
import subprocess
import librosa
import librosa.display
import IPython.display as ipd
import csv
import pandas as pd
import ast 
from pathlib import Path, PurePath   
from tqdm.notebook import tqdm

###Utility functions from [AudioSignals.ipynb](https://github.com/lucamaiano/ADM/blob/master/2021/Homework_4/AudioSignals.ipynb):

In [3]:
def convert_mp3_to_wav(audio:str) -> str:  
    """Convert an input MP3 audio track into a WAV file.

    Args:
        audio (str): An input audio track.

    Returns:
        [str]: WAV filename.
    """
    if audio[-3:] == "mp3":
        wav_audio = audio[:-3] + "wav"
        if not Path(wav_audio).exists():
                subprocess.check_output(f"ffmpeg -i {audio} {wav_audio}", shell=True)
        return wav_audio
    
    return audio

def plot_spectrogram_and_peaks(track:np.ndarray, sr:int, peaks:np.ndarray, onset_env:np.ndarray) -> None:
    """Plots the spectrogram and peaks 

    Args:
        track (np.ndarray): A track.
        sr (int): Aampling rate.
        peaks (np.ndarray): Indices of peaks in the track.
        onset_env (np.ndarray): Vector containing the onset strength envelope.
    """
    times = librosa.frames_to_time(np.arange(len(onset_env)),
                            sr=sr, hop_length=HOP_SIZE)

    plt.figure()
    ax = plt.subplot(2, 1, 2)
    D = librosa.stft(track)
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(D), ref=np.max),
                            y_axis='log', x_axis='time')
    plt.subplot(2, 1, 1, sharex=ax)
    plt.plot(times, onset_env, alpha=0.8, label='Onset strength')
    plt.vlines(times[peaks], 0,
            onset_env.max(), color='r', alpha=0.8,
            label='Selected peaks')
    plt.legend(frameon=True, framealpha=0.8)
    plt.axis('tight')
    plt.tight_layout()
    plt.show()

def load_audio_peaks(audio, offset, duration, hop_size):
    """Load the tracks and peaks of an audio.

    Args:
        audio (string, int, pathlib.Path or file-like object): [description]
        offset (float): start reading after this time (in seconds)
        duration (float): only load up to this much audio (in seconds)
        hop_size (int): the hop_length

    Returns:
        tuple: Returns the audio time series (track) and sampling rate (sr), a vector containing the onset strength envelope
        (onset_env), and the indices of peaks in track (peaks).
    """
    try:
        track, sr = librosa.load(audio, offset=offset, duration=duration)
        onset_env = librosa.onset.onset_strength(track, sr=sr, hop_length=hop_size)
        peaks = librosa.util.peak_pick(onset_env, 10, 10, 10, 10, 0.5, 0.5)
    except Error as e:
        print('An error occurred processing ', str(audio))
        print(e)

    return track, sr, onset_env, peaks

### Implemented functions:


In [4]:
def shingling_matrix(dataframe, feature_index):
  n=dataframe.shape[0]
  peaks_list=[]
  for i in range(0,n):
    peaks_list=list(set(peaks_list)|set(dataframe.iloc[i,feature_index]))
  p=len(peaks_list)
  shingling=pd.DataFrame(np.zeros((p,n)),dtype=int)
  for i in tqdm(range(0,p)):
    for j in range(0,n):
      if peaks_list[i] in dataframe.iloc[j,feature_index]:
        shingling.iloc[i,j]=1
  return peaks_list, shingling



def signature_matrix(shinglingmatrix,num_permutation):
    permutations=[]
    n=shinglingmatrix.shape[1]
    signature = pd.DataFrame(np.zeros((num_permutation,n),dtype=int))
    for i in tqdm(range(0,num_permutation)):
        p = np.random.permutation(len(shinglingmatrix))
        permutations.append(p)
        s = shinglingmatrix.iloc[p]
        for j in range(0,n):
            index = np.where(np.array(s.iloc[:,j])==1)[0][0]
            signature.iloc[i,j] = int(index)
    
    return permutations, signature

def minhashing(signaturematrix, num_band):
    buckets = {}
    num_perm = signaturematrix.shape[0]
    num_doc = signaturematrix.shape[1]
    row_for_band = num_perm/num_band
    for i in range(0,num_band):
        for j in range(0,num_doc):
            key=tuple(signaturematrix.iloc[int(i*row_for_band):int(row_for_band+(i*row_for_band)),j])
            if key not in buckets.keys():
                buckets[key]=[j]
            else:
                buckets[key].append(j)
    return buckets


def compute_query(query_wav_directory, modelli_csv_directory, peaks_list, permutations_list, buckets):

    df = pd.read_table(modelli_csv_directory, sep=',', converters = {'Title':str,'Peaks':ast.literal_eval} )

    _, _, query_onset_env, query_peaks = load_audio_peaks(query_wav_directory, OFFSET, DURATION, HOP_SIZE)

     
    query_peaks = list(query_onset_env[query_peaks])
    query_peaks = np.array([round(x,1) for x in query_peaks])
    shingling_query = np.zeros(len(peaks_list),dtype=int)

    for i in range(len(peaks_list)):
        if peaks_list[i] in query_peaks:
            shingling_query[i] = 1
    
    num_perm = len(permutations_list)
    
    signature_query = np.zeros(num_perm,dtype=int)
    

    for i in range(num_perm):
        hash = permutations_list[i]
        index = np.where(shingling_query[hash]==1)[0][0]
        signature_query[i] = index
    

    num_row_for_band = len(list(buckets.keys())[0])
    num_band = int(num_perm/num_row_for_band)

    set_doc = set()



    for i in range(num_band):
        key = signature_query[int(i*num_row_for_band):int(num_row_for_band+(i*num_row_for_band))]
        try :
            set_doc.update(set(buckets[tuple(key)]))
        except KeyError:
            continue 
    
    set_doc = list(set_doc)

    if len(set_doc)==1:
        return  str(df.iloc[set_doc[0],0]).replace('_',' ')

    else :
        return np.array(df.iloc[set_doc,0],dtype= str)



    




###Global settings
Those are the parameters for the conversion from  file.mp:3 to file.wav and from file.wav to peak detection:

- *N_TRACKS*, number of tracks to read.
- *HOP_SIZE*.
- *OFFSET*, time (in seconds) before reading the file.
- *DURATION*, window of reading. 



In [5]:
N_TRACKS = 1413
HOP_SIZE = 512
OFFSET = 1.0
DURATION = 30 


###Preprocessing:  
we are converting our data from *file.mp3* to *file.wav*.

In [6]:
data_folder = Path("/content/drive/MyDrive/ADM-HW04/MP3")
mp3_tracks = data_folder.glob("*/*/*.mp3")

In [ ]:
for track in tqdm(mp3_tracks, total=N_TRACKS):
    convert_mp3_to_wav(str(track))

Now we create a file.csv containing all the informations needed for the implementation of the algorithm.

In [7]:
tracks = data_folder.glob("*/*/*.wav")
data_csv_directory = "/content/drive/MyDrive/ADM-HW04/data.csv"

In [102]:
with open(data_csv_directory, "w", encoding='utf-8', newline='') as file:
  writer = csv.writer(file)
  
  header = ['Title','Peaks']

  writer.writerow(header)

  for idx, audio in tqdm(enumerate(tracks),total=N_TRACKS):
    track, sr, onset_env, peaks = load_audio_peaks(audio, OFFSET, DURATION, HOP_SIZE)
    title = str(audio).split('/')[-1]
    title = title[3:len(title)-4]
    peaks = list(onset_env[peaks])
    peaks = [round(x,1) for x in peaks]
    data = [title, peaks]
    print(data)
    writer.writerow(data)


  0%|          | 0/1413 [00:00<?, ?it/s]

['Make_It', [4.2, 3.6, 6.9, 5.0, 5.1, 7.9, 5.4, 4.3, 4.4, 4.5, 5.1, 4.9, 4.0, 4.7, 3.8, 4.1, 4.5, 4.3, 3.0, 4.5, 6.2, 5.0, 3.7, 4.3, 4.7, 4.3, 5.1, 4.9, 4.8, 3.4, 5.9, 4.3, 3.3, 5.2, 5.4, 5.6, 3.0, 4.1, 2.6, 4.9, 4.9, 5.6, 5.8, 4.9, 5.0, 5.7, 4.9, 4.0, 5.3, 5.4, 5.6, 5.5, 5.6, 4.7, 5.3, 4.2, 4.7, 5.1, 6.0, 4.6, 1.9, 1.6, 1.9, 2.5, 4.0, 1.8]]
['Somebody', [14.0, 3.5, 4.3, 6.3, 10.9, 4.0, 13.4, 3.5, 5.6, 3.3, 4.8, 3.9, 16.1, 3.8, 7.9, 7.7, 7.4, 3.5, 10.0, 3.4, 6.8, 7.3, 7.5, 2.2, 2.8, 4.5, 4.1, 2.6, 5.2, 2.6, 3.7, 2.7, 3.9, 4.7, 4.8, 6.5, 3.7, 5.5, 9.4, 5.0, 9.2, 4.6, 5.5, 4.7, 3.3, 4.4, 7.1, 5.5, 2.6, 4.0, 3.9, 4.7, 2.7, 4.3, 2.4, 2.9, 2.9, 3.2, 4.5]]
['Dream_On', [3.2, 2.5, 3.7, 2.2, 3.9, 2.9, 4.1, 2.1, 3.8, 2.8, 3.9, 2.6, 3.8, 3.2, 4.6, 2.0, 4.6, 2.8, 3.7, 2.1, 4.1, 3.4, 5.0, 1.6, 4.0, 2.6, 4.2, 5.9, 2.9, 4.8, 2.3, 7.1, 2.5, 2.5, 2.8, 4.3, 1.8, 3.9, 2.8, 3.2, 1.9, 3.4, 3.0, 6.2, 2.8, 3.9, 2.5, 4.2, 2.0, 3.5, 3.2, 5.4, 1.5, 3.5, 1.9, 4.7, 4.1, 1.5, 5.2, 6.4, 1.5, 4.3, 2.1, 5.0, 5.1, 1.

In [8]:
df = pd.read_table(data_csv_directory,sep=',',converters={'Title':str,'Peaks':ast.literal_eval})
df

,Title,Peaks
0,Make_It,"[4.2, 3.6, 6.9, 5.0, 5.1, 7.9, 5.4, 4.3, 4.4, ..."
1,Somebody,"[14.0, 3.5, 4.3, 6.3, 10.9, 4.0, 13.4, 3.5, 5...."
2,Dream_On,"[3.2, 2.5, 3.7, 2.2, 3.9, 2.9, 4.1, 2.1, 3.8, ..."
3,One_Way_Street,"[4.9, 13.0, 3.4, 11.7, 6.9, 4.1, 2.1, 17.1, 12..."
4,Mama_Kin,"[6.9, 6.8, 4.5, 8.0, 3.0, 3.7, 2.6, 7.2, 8.2, ..."
...,...,...
1408,The_Refugee,"[7.9, 8.4, 10.7, 9.5, 7.0, 10.3, 4.0, 6.9, 6.5..."
1409,Two_Hearts_Beat_As_One,"[3.5, 7.8, 4.2, 8.2, 11.7, 13.0, 13.9, 3.7, 10..."
1410,Red_Light,"[6.0, 11.6, 2.0, 3.5, 6.6, 5.6, 8.4, 2.3, 3.2,..."
1411,Surrender,"[4.3, 3.5, 3.9, 2.6, 3.0, 3.6, 8.1, 2.9, 3.2, ..."


### Passages of the algorithm: 
The  first passages consist  the building the structures of the algorithm in order to habe the possibility to compute the query.
First of all from the dataframe that we created before we take all the extracted peacks in order to create the signature matrix of our set of documents.

In [11]:
peaks_list, shingling = shingling_matrix(df, 1)

  0%|          | 0/304 [00:00<?, ?it/s]

Once we obtained our shingling matrix we need to choose a class of hash funtions from which we can take our b-random hash functions to detrmined the fingerprints of our documents.  

In our case the family from we choose our functions is the permutations of the signature matrix n-times and  each time for each document we take the index of the first row that has value different form  zero. This process should give us a new matrix with the same columns (one for each document) and number of rows equal to n.

In [17]:
NUM_PERM = 60

In [18]:
permutations, signature = signature_matrix(shingling,NUM_PERM)

  0%|          | 0/60 [00:00<?, ?it/s]

The given matrix is named signature matrix from which we are taking our finger print.
In fact  we assign as finger print fo $doc_i$, $h(doc_i)$, the first r-rows of the matrix on the column's document. We do this b-times such that $b*r=n$ so we will have b fingerprints for each document.  

It may happen that some documents could have the same finger print, this is called collapse;this depends on the prameter n and b that we choose beacuse took random an hash function from my family the probability that the given hash fucntions would givethe same results when applied to two different documents, it s equal  to the jacard similarity of the given documents.
In other word, similar documents have more chace to have the same finger print, in particular the minimum similarity (thrashold) that two documents have to have in order to have the same fingerprint is given by  $TH=(1/b)^{(1/r)}$. So depending on those parameters we will have different levels of accuracy  and  false positive rate.

This result is the keystone for the  computation of the query.

In [19]:
NUM_BANDS = 4
NUM_OF_ROWS = 15
TH = (1/NUM_BANDS)**(1/NUM_OF_ROWS)
TH

0.9117224885582168

In [22]:
buckets = minhashing(signature, NUM_BANDS)
buckets;

What we have done is storing  all the fingerprints as keys of a vocabulary and the associated documents as values of that key.  

Once we have all the structures of our algorithm we need only to compute the query. The first thing to do is to pre-process the query  in the same way we did with the documents; then once we obtain its peaks we have to build the shingling vector for the query and its corresponding signature vector according to the permutations that we used for the building.
Now with the same process descripted before we detrmine the finger prints of the query and search in our dictionary the values for thegiven keys (if the key exists). 
Now there are possibility:
- Any keys exist, so there are no similarity between the query and our documents (under the hipotesys that we did everything  correct).
- At least one key ping more than one document, false positive case. In order to resolve this we  can compute a more  sensible  similarity between the query and the given documents, than choose the best result.
- All the existing keys  ping the same document. We coputed the query!!! 

In [24]:
for i in range(1,11):
        wav = '/content/drive/MyDrive/ADM-HW04/Query/track' + str(i) + '.wav'
        titolo = compute_query(query_wav_directory = wav, modelli_csv_directory = data_csv_directory, peaks_list = peaks_list, permutations_list = permutations , buckets = buckets)
        print('track' + str(i)+ ': '+str(titolo))

track1: Dream On
track2: I Want To Break Free
track3: October
track4: Ob-La-Di Ob-La-Da
track5: Karma Police
track6: Heartbreaker
track7: Go Your Own Way
track8: American Idiot
track9: Somebody
track10: Black Friday
